In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [2]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

/opt/anaconda3/envs/gpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
examples = [
    {
        "question": "Tell me about the movie, Avengers: Endgame",
        "answer": """
        Director: Anthony Russo, Joe Russo
        Main Cast: Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson
        Budget: Approximately $350 million
        Box Office Revenue: Approximately $2.79 billion worldwide
        Genre: Superhero, Action, Sci-Fi
        Brief Synopsis: After Thanos wipes out half of all life in the universe, the surviving Avengers come together for one final battle to undo the devastation and bring back their lost loved ones.
        """    
     },
     {
        "question": "Tell me about the movie, the Lion King",
        "answer":"""
        Director: Jon Favreau
        Main Cast: Donald Glover, Beyoncé, James Earl Jones, Chiwetel Ejiofor
        Budget: Approximately $260 million
        Box Office Revenue: Approximately $1.65 billion worldwide
        Genre: Animation, Drama, Adventure
        Brief Synopsis: Young lion Simba flees his kingdom after the death of his father, Mufasa, but later embraces his destiny and returns to reclaim his throne.
        """
     },
     {
        "question": "Tell me about the movie, Top Gun: Maverick",
        "answer": """
        Director: Joseph Kosinski
        Main Cast: Tom Cruise, Miles Teller, Jennifer Connelly
        Budget: Approximately $170 million
        Box Office Revenue: Approximately $1.48 billion worldwide
        Genre: Action, Drama
        Brief Synopsis: Legendary pilot Maverick returns as a flight instructor after 30 years to train a new generation of pilots while confronting past traumas.
        """
     }
]

In [4]:
example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Tell me about the movie {movie}",
    input_variables=["movie"],
)

chain = prompt | chat

chain.invoke({"movie":"Dark Knight"})

AI:
        Director: Christopher Nolan
        Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine, Gary Oldman
        Budget: Approximately $185 million
        Box Office Revenue: Approximately $1.005 billion worldwide
        Genre: Superhero, Action, Crime
        Brief Synopsis: Batman faces off against the Joker, a criminal mastermind who wreaks havoc on Gotham City, while also dealing with the rise of Two-Face, a former ally turned enemy.

AIMessage(content='AI:\n        Director: Christopher Nolan\n        Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine, Gary Oldman\n        Budget: Approximately $185 million\n        Box Office Revenue: Approximately $1.005 billion worldwide\n        Genre: Superhero, Action, Crime\n        Brief Synopsis: Batman faces off against the Joker, a criminal mastermind who wreaks havoc on Gotham City, while also dealing with the rise of Two-Face, a former ally turned enemy.', response_metadata={'finish_reason': 'stop'}, id='run-b58042ba-994e-49f2-8b7e-91d888fa1995-0')

In [5]:
chain.invoke({"movie":"Moana"})

AI:
        Director: Ron Clements, John Musker
        Main Cast: Auli'i Cravalho, Dwayne Johnson
        Budget: Approximately $150 million
        Box Office Revenue: Approximately $690 million worldwide
        Genre: Animation, Adventure, Comedy
        Brief Synopsis: Moana, a spirited teenager, sets sail on a daring mission to save her people and discovers her own identity along the way with the help of the demigod Maui.

AIMessage(content="AI:\n        Director: Ron Clements, John Musker\n        Main Cast: Auli'i Cravalho, Dwayne Johnson\n        Budget: Approximately $150 million\n        Box Office Revenue: Approximately $690 million worldwide\n        Genre: Animation, Adventure, Comedy\n        Brief Synopsis: Moana, a spirited teenager, sets sail on a daring mission to save her people and discovers her own identity along the way with the help of the demigod Maui.", response_metadata={'finish_reason': 'stop'}, id='run-c595dc00-c9fc-4036-bba8-26e440da7c0a-0')

In [6]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

In [7]:
examples = [
    {
        "movie": "Avengers: Endgame",
        "answer": """
        Director: Anthony Russo, Joe Russo
        Main Cast: Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson
        Budget: Approximately $350 million
        Box Office Revenue: Approximately $2.79 billion worldwide
        Genre: Superhero, Action, Sci-Fi
        Brief Synopsis: After Thanos wipes out half of all life in the universe, the surviving Avengers come together for one final battle to undo the devastation and bring back their lost loved ones.
        """    
     },
     {
        "movie": "The Lion King",
        "answer":"""
        Director: Jon Favreau
        Main Cast: Donald Glover, Beyoncé, James Earl Jones, Chiwetel Ejiofor
        Budget: Approximately $260 million
        Box Office Revenue: Approximately $1.65 billion worldwide
        Genre: Animation, Drama, Adventure
        Brief Synopsis: Young lion Simba flees his kingdom after the death of his father, Mufasa, but later embraces his destiny and returns to reclaim his throne.
        """
     },
     {
        "movie": "Top Gun: Maverick",
        "answer": """
        Director: Joseph Kosinski
        Main Cast: Tom Cruise, Miles Teller, Jennifer Connelly
        Budget: Approximately $170 million
        Box Office Revenue: Approximately $1.48 billion worldwide
        Genre: Action, Drama
        Brief Synopsis: Legendary pilot Maverick returns as a flight instructor after 30 years to train a new generation of pilots while confronting past traumas.
        """
     }
]

In [8]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","Do you know about the movie {movie}?"),
        ("ai","{answer}"),
    ],
)

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a Ph.D degree in Movie. And you are insane at movie."),
        fewshot_prompt,
        ("human", "Do you know about the movie {movie}?"),
    ]
)

final_chain = final_prompt | chat

final_chain.invoke({"movie":"The Dark Knight"})

Director: Christopher Nolan
Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart, Maggie Gyllenhaal
Budget: Approximately $185 million
Box Office Revenue: Approximately $1.005 billion worldwide
Genre: Superhero, Crime, Drama
Brief Synopsis: Batman faces his greatest challenge yet as he battles the Joker, a sadistic criminal mastermind who wreaks havoc on Gotham City. Ledger's portrayal of the Joker earned him a posthumous Academy Award for Best Supporting Actor.

AIMessage(content="Director: Christopher Nolan\nMain Cast: Christian Bale, Heath Ledger, Aaron Eckhart, Maggie Gyllenhaal\nBudget: Approximately $185 million\nBox Office Revenue: Approximately $1.005 billion worldwide\nGenre: Superhero, Crime, Drama\nBrief Synopsis: Batman faces his greatest challenge yet as he battles the Joker, a sadistic criminal mastermind who wreaks havoc on Gotham City. Ledger's portrayal of the Joker earned him a posthumous Academy Award for Best Supporting Actor.", response_metadata={'finish_reason': 'stop'}, id='run-5a3656bf-ad05-4ac5-b725-cc4fe733e11a-0')

In [9]:
final_chain.invoke({"movie":"Moana"})


        Director: Ron Clements, John Musker
        Main Cast: Auli'i Cravalho, Dwayne Johnson
        Budget: Approximately $150 million
        Box Office Revenue: Approximately $690 million worldwide
        Genre: Animation, Adventure, Comedy
        Brief Synopsis: Moana, a spirited young girl, sets sail on a daring mission to save her people and discovers her own identity along the way with the help of the demigod Maui.
        

AIMessage(content="\n        Director: Ron Clements, John Musker\n        Main Cast: Auli'i Cravalho, Dwayne Johnson\n        Budget: Approximately $150 million\n        Box Office Revenue: Approximately $690 million worldwide\n        Genre: Animation, Adventure, Comedy\n        Brief Synopsis: Moana, a spirited young girl, sets sail on a daring mission to save her people and discovers her own identity along the way with the help of the demigod Maui.\n        ", response_metadata={'finish_reason': 'stop'}, id='run-efc21fde-d113-43b5-a6ba-49a9a6b523c6-0')